In [7]:
import control as ct
import numpy as np

In [8]:
# define state space matrices
A = [[0, 1], [-1, -1]]
B = [[0], [1]]
C = [[1, 0]]
D = [[0]]

# create the system
sys = ct.ss(A, B, C, D)
sys

<LinearIOSystem:sys[4]:['u[0]']->['y[0]']>

In [12]:
obs = ct.obsv(sys.A, sys.C)
np.linalg.matrix_rank(obs)

2

In [16]:
# x = [ptx, pty, pux_dot, puy_dot, pbx_dot, pby_dot]
# ptx_dot = pux_dot + pbx_dot
# pty_dot = puy_dot + pby_dot

A = [[0, 0, 1, 0, 1, 0],
     [0, 0, 0, 1, 0, 1],
     [0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0]]
# measurements of pt and pt_dot are taken
# y = [ptx pty ptx_dot pty_dot]
C = [[1, 0, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [0, 0, 1, 0, 1, 0],
     [0, 0, 0, 1, 0, 1],
     [0, 0, 1, 0, 0, 0],
     [0, 0, 0, 1, 0, 0]]

obs = ct.obsv(A, C)
np.linalg.matrix_rank(obs)

6